
# 4️⃣ Modelo Supervisado de **Churn Valioso K-Means**

En este notebook vamos a construir un **modelo supervisado** para predecir la probabilidad de que un cliente pertenezca a la clase **`Churn_Valioso_KMeans = 1`**.

## Objetivos

1. Cargar el dataset procesado (`superstore_procesado.csv`) y **crear/revisar** la variable objetivo de churn valioso.
2. Definir el conjunto de variables (features) para el modelo de churn valioso.
3. Preparar un **pipeline de preprocesamiento** (imputación, escalado, one-hot encoding).
4. Entrenar y comparar dos modelos:
   - **Regresión logística** (con regularización L1 / L2).
   - **Random Forest**.
5. Usar **GridSearchCV** con validación cruzada estratificada y métrica **ROC-AUC**.
6. Evaluar el mejor modelo en un conjunto de validación (hold-out).
7. Analizar la **importancia de variables** (coeficientes o feature importances).
8. Guardar el pipeline completo entrenado en `../models/churn_pipeline.pkl`.


## 1. Carga de datos y creación de la variable objetivo

In [ ]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

# Configuración visual
plt.style.use("seaborn-v0_8")
sns.set(font_scale=1.0)

data_path = Path("../data/superstore_procesado.csv")
df = pd.read_csv(data_path)

print("Columnas del dataset:")
print(df.columns)

df.head()


In [ ]:

# --- Definición de Churn Valioso K-Means (por si NO viene ya en el CSV) ---
# Definición tomada del notebook de CDA:
# - Inactivo: Recency >= 83 (umbral del cluster más inactivo en K-Means)
# - Valioso: CLV_log >= mediana
# - Churn_Valioso_KMeans = 1 si se cumplen ambas condiciones

UMBRAL_RECENCY_KMEANS = 83

if "Churn_Valioso_KMeans" not in df.columns:
    if "CLV_log" not in df.columns:
        # Si por alguna razón no existe CLV_log, lo recalculamos
        df["CLV_log"] = np.log1p(df["MntTotal"]) * np.log1p(df["TotalPurchases"])
    
    mediana_clv = df["CLV_log"].median()

    df["Churn_Valioso_KMeans"] = (
        (df["Recency"] >= UMBRAL_RECENCY_KMEANS) &
        (df["CLV_log"] >= mediana_clv)
    ).astype(int)

    # Opcional: churn simple solo por recency, útil para análisis comparativo
    df["Churn_KMeans"] = (df["Recency"] >= UMBRAL_RECENCY_KMEANS).astype(int)

    print("✅ Columnas de churn creadas desde la definición original de CDA.")
else:
    print("✅ Columnas de churn ya presentes en el dataset.")


## 2. Distribución de la variable objetivo

In [ ]:

target_col = "Churn_Valioso_KMeans"

tasa_churn = (df[target_col].value_counts(normalize=True).round(4) * 100)
print("Distribución de Churn_Valioso_KMeans (%):")
print(tasa_churn)



La celda anterior muestra el **porcentaje de clientes churn valioso (1)** frente a **no churn valioso (0)**.
Normalmente, el churn valioso será un porcentaje pequeño del total de clientes, lo que implica un:

> **Problema de clases desbalanceadas** → hay muchos más ejemplos de clase 0 que de clase 1.


In [ ]:

# Gráfico de barras de la distribución de la clase
fig, ax = plt.subplots(figsize=(5,4))
sns.barplot(
    x=tasa_churn.index.astype(str),
    y=tasa_churn.values,
    ax=ax
)
ax.set_title("Distribución de Churn Valioso K-Means")
ax.set_xlabel("Churn_Valioso_KMeans")
ax.set_ylabel("% de clientes")
plt.show()


## 3. Selección de variables (features) y target

In [ ]:

# Features seleccionadas para el modelo supervisado
feature_cols = [
    "Recency", "MntTotal", "TotalPurchases", "Income",
    "Perc_CatalogPurchases", "NumWebVisitsMonth",
    "Kidhome", "Teenhome",
    "Education", "Marital_Status"
]

X = df[feature_cols].copy()
y = df[target_col].copy()

X.head()



**Nota:** No incluimos `CLV_log` como feature porque forma parte directa de la definición de churn valioso
y generaría **data leakage**.


## 4. Split train / validación

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_val.shape



Usamos un 80% de los datos para entrenamiento y 20% para validación, estratificando por la variable objetivo.


## 5. Pipeline de preprocesamiento

In [ ]:

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_features = [
    "Recency", "MntTotal", "TotalPurchases", "Income",
    "Perc_CatalogPurchases", "NumWebVisitsMonth",
    "Kidhome", "Teenhome"
]
categorical_features = ["Education", "Marital_Status"]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

preprocess


## 6. Modelos candidatos y GridSearchCV

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV

base_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced", solver="liblinear"))
])

param_grid = [
    {
        "model": [LogisticRegression(max_iter=1000, class_weight="balanced", solver="liblinear")],
        "model__penalty": ["l1", "l2"],
        "model__C": [0.1, 1.0, 10.0],
    },
    {
        "model": [RandomForestClassifier(class_weight="balanced", random_state=42)],
        "model__n_estimators": [100, 300],
        "model__max_depth": [None, 5, 10],
        "model__min_samples_split": [2, 5],
    }
]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Modelos que se probarán en GridSearch:")
for params in param_grid:
    for m in params["model"]:
        print(" -", type(m).__name__)


In [ ]:

grid = GridSearchCV(
    estimator=base_pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1
)

grid


## 7. Entrenamiento y selección del mejor modelo

In [ ]:

grid.fit(X_train, y_train)
best_model = grid.best_estimator_

print("Mejores hiperparámetros encontrados:")
print(grid.best_params_)

best_model_name = type(best_model.named_steps["model"]).__name__
print("\nMejor modelo seleccionado:", best_model_name)


## 8. Evaluación en el conjunto de validación

In [ ]:

from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, RocCurveDisplay

y_val_proba = best_model.predict_proba(X_val)[:, 1]
y_val_pred = best_model.predict(X_val)

roc = roc_auc_score(y_val, y_val_proba)
print(f"ROC-AUC (validación): {roc:.4f}\n")

print("Matriz de confusión (validación):")
cm = confusion_matrix(y_val, y_val_pred)
print(cm)

print("\nReporte de clasificación (validación):")
print(classification_report(y_val, y_val_pred, digits=4))


In [ ]:

fig, ax = plt.subplots(figsize=(6,5))
RocCurveDisplay.from_predictions(y_val, y_val_proba, ax=ax)
ax.set_title("Curva ROC - Validación")
plt.show()


In [ ]:

fig, ax = plt.subplots(figsize=(4,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, ax=ax)
ax.set_xlabel("Predicción")
ax.set_ylabel("Real")
ax.set_title("Matriz de confusión - Validación")
plt.show()


## 9. Importancia de variables

In [ ]:

ohe = best_model.named_steps["preprocess"].named_transformers_["cat"].named_steps["ohe"]
num_features = numeric_features
cat_features = list(ohe.get_feature_names_out(categorical_features))
all_features = num_features + cat_features

best_model_name = type(best_model.named_steps["model"]).__name__
print("Mejor modelo:", best_model_name)

import pandas as pd

if best_model_name == "LogisticRegression":
    model = best_model.named_steps["model"]
    coef = model.coef_[0]
    feat_imp = pd.DataFrame({
        "feature": all_features,
        "importance": coef
    })
    feat_imp["abs_importance"] = feat_imp["importance"].abs()
    feat_imp = feat_imp.sort_values("abs_importance", ascending=False).head(20)
    
    print("\nTop 20 coeficientes (LogisticRegression):")
    print(feat_imp[["feature", "importance"]])
    
    fig, ax = plt.subplots(figsize=(8,6))
    sns.barplot(data=feat_imp, x="abs_importance", y="feature", ax=ax)
    ax.set_title("Importancia de variables (|coeficientes|)")
    ax.set_xlabel("|Coeficiente|")
    ax.set_ylabel("Feature")
    plt.show()

elif best_model_name == "RandomForestClassifier":
    model = best_model.named_steps["model"]
    importances = model.feature_importances_
    feat_imp = pd.DataFrame({
        "feature": all_features,
        "importance": importances
    }).sort_values("importance", ascending=False).head(20)
    
    print("\nTop 20 importancias (RandomForest):")
    print(feat_imp)
    
    fig, ax = plt.subplots(figsize=(8,6))
    sns.barplot(data=feat_imp, x="importance", y="feature", ax=ax)
    ax.set_title("Importancia de variables (Random Forest)")
    ax.set_xlabel("Importancia")
    ax.set_ylabel("Feature")
    plt.show()


## 10. Guardado del pipeline entrenado

In [ ]:

import joblib

models_path = Path("../models")
models_path.mkdir(exist_ok=True)

model_path = models_path / "churn_pipeline.pkl"
joblib.dump(best_model, model_path)
print(f"✅ Modelo guardado en: {model_path}")
